In [36]:
from bs4 import BeautifulSoup
import os
import requests
import pandas as pd
import itertools
import re
import numpy as np
import urllib3
import warnings
warnings.filterwarnings("ignore")
import plotly.express as px

# 以搜尋結果面網址
headers = {
        'Connection': 'keep-alive',
        'Pragma': 'no-cache',
        'Cache-Control': 'no-cache',
        'DNT': '1',
        'Upgrade-Insecure-Requests': '1',
        # You may want to change the user agent if you get blocked
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.113 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',

        'Referer': 'https://www.booking.com/index.en-gb.html',
    }
numberHotel = 0
error = 0
n = 0
df = pd.DataFrame(columns=['name','address', 'distance', 'rating', 'comment', 'price'])
while True:
    url = f'https://www.booking.com/searchresults.zh-tw.html?ss=taipei&checkin=2023-12-01&checkout=2023-12-02&group_adults=2&selected_currency=TWD&lang=en-us&soz=1&order=distance_from_search&offset={numberHotel}'
    response = requests.get(url=url, headers=headers)
    soup = BeautifulSoup(response.text, 'lxml')
    hotel_items = soup.find_all('div', {'data-testid': 'property-card-container'})  # 假设每个酒店条目都有这个类名

    for item in hotel_items:
        name_element = item.find('div', {'data-testid': 'title'})  # 查找酒店名称
        address_element = item.find('span', {'data-testid': 'address'})
        distance_element = item.find('span', {'data-testid': 'distance'})  # 查找距离
        rating_element = item.find('div', {'class': "a3b8729ab1 d86cee9b25"})  # 查找评分
        comment_element = item.find('div', {'class': 'a3b8729ab1 e6208ee469 cb2cbb3ccb'})
        price_element = item.find('span', {'data-testid': 'price-and-discounted-price'})
        if name_element:
            try:
                name = name_element.text.strip()
                address = address_element.text.strip()
                distance = distance_element.text[0:5].strip()
                rating = rating_element.text.strip()
                comment = comment_element.text.strip()
                price = price_element.text.strip()
                data_dict = {'name': name,'address':address, 'distance': distance, 'rating': rating, 'comment': comment, 'price': price}
                df = df.append(data_dict, ignore_index=True)
                n += 1
            except:
                error += 1

        else:
            break
    try:
        totalHotel = int(soup.find('div', {'class': 'd8f77e681c'}).text.strip()[-20:-17])  # 假设每个酒店条目都有这个类名
    except:
        break
    numberHotel += 25
    if n + error == totalHotel:
        break
    # if numberHotel == 200:
    #     break
    print(df.info())

df.to_csv('hotel.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   name      24 non-null     object
 1   address   24 non-null     object
 2   distance  24 non-null     object
 3   rating    24 non-null     object
 4   comment   24 non-null     object
 5   price     24 non-null     object
dtypes: object(6)
memory usage: 1.2+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   name      49 non-null     object
 1   address   49 non-null     object
 2   distance  49 non-null     object
 3   rating    49 non-null     object
 4   comment   49 non-null     object
 5   price     49 non-null     object
dtypes: object(6)
memory usage: 2.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74 entries, 0 to 73
Data columns (total 6 column

In [37]:
import pandas as pd
def clean_distance(value):
    try:
        if 'k' in value:
            value = value.replace('km', '')
            value = value.replace('k', '')
            return float(value.strip())
        else:
            value = value.replace('m', '')
            return float(value.strip()) / 1000
    except:
        print(value)
        return np.nan
    
def clean_price(value):
    return int(value.replace('TWD', '').replace(',', ''))

df = pd.read_csv("hotel.csv")
df['distance'] = df['distance'].apply(clean_distance)
df['price'] = df['price'].apply(clean_price)

In [38]:
df

,Unnamed: 0,name,address,distance,rating,comment,price
0,0,Inn Cube - Taipei Main Station,"Zhongzheng District, Taipei",0.1000,7.6,Good,2340
1,1,Taiwan Youth Hostel & Capsule Hotel,"Zhongzheng District, Taipei",0.1500,8.7,Excellent,2300
2,2,NYS Loft Hotel,"Zhongzheng District, Taipei",0.1500,8.1,Very Good,2640
3,3,Mayer Inn,"Zhongzheng District, Taipei",0.2000,7.8,Good,4860
4,4,Water meworld,"Zhongzheng District, Taipei",0.2000,7.9,Good,4085
...,...,...,...,...,...,...,...
468,468,Grand HiLai Taipei,"Nangang District, Taipei",0.0103,8.1,Very Good,7472
469,469,Great Family Hotel,"Beitou District, Taipei",0.0104,8.1,Very Good,2480
470,470,Jeanette Home,Taipei,0.0104,9.6,Exceptional,3970
471,471,CHECK inn Select Taipei Yamgmingshan,"Shilin District , Taipei",0.0113,7.4,Good,6036


In [39]:

fig = px.scatter(df, x='price', y='distance', hover_data=['name','comment'], color='rating')
fig.update_traces(hovertemplate='Name: %{customdata[0]}<br>Distance: %{y}<br>Price: %{x}<br>Comment: %{customdata[1]}')
fig.show()
